<a href="https://colab.research.google.com/github/jarekwan/jarwan_projekt/blob/main/Projekt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

BIBLIOTEKI I POBRANIE JSON Z RESTCOUNTRIES

In [2]:
import requests      # biblioteka do pobierania danych z API
import pandas as pd  # biblioteka do pracy z tabelami (DataFrame)
import json
from typing import List, Dict, Optional
from pydantic import BaseModel, ValidationError  # Pydantic do walidacji danych
# adres API RestCountries z wybranymi polami (max 10 na raz)
url = "https://restcountries.com/v3.1/all?fields=name,capital,region,subregion,population,area,currencies,languages,timezones,flags"

# pobranie danych z API
resp = requests.get(url)


#sprawdzanie surowego jsona
# wypisanie statusu odpowiedzi (200 = OK)
print("Status:", resp.status_code)

print(len(resp.text))#po nim wstawiam PYDANTIC
print(resp.text[:500])

# czy zaweira kluczowe frazy
print("name" in resp.text)
print("population" in resp.text)


Status: 200
170660
[{"flags":{"png":"https://flagcdn.com/w320/jm.png","svg":"https://flagcdn.com/jm.svg","alt":"The flag of Jamaica is divided by a gold diagonal cross into four alternating triangular areas of green at the top and bottom, and black on the hoist and fly sides"},"name":{"common":"Jamaica","official":"Jamaica","nativeName":{"eng":{"official":"Jamaica","common":"Jamaica"},"jam":{"official":"Jamaica","common":"Jamaica"}}},"currencies":{"JMD":{"name":"Jamaican dollar","symbol":"$"}},"capital":["Kingston
True
True


SPRAWDZANIE CZY JSON POBRAL SIE PRAWIDLOWO

In [3]:
if resp.status_code == 200:
    # konwersja odpowiedzi JSON → Python (lista słowników)
    data = resp.json()
    print(type(data))
    # sprawdź typ pierwszego elementu
    print(type(data[0]))
    # zamiana listy słowników na tabelę Pandas



else:
    # jeśli status ≠ 200, wyświetlamy komunikat błędu
    print("Błąd:", resp.text)

<class 'list'>
<class 'dict'>


SPRAWDZENIE 1SZY ELEMNTOW POSZCZEGOLNYCH POL JAKIEGO SA TYPU

In [4]:
print(type(data[0]["name"]))
print(type(data[0]["capital"]))
print(type(data[0]["region"]))
print(type(data[0]["subregion"]))
print(type(data[0]["population"]))
print(type(data[0]["area"]))
print(type(data[0]["currencies"]))
print(type(data[0]["languages"]))
print(type(data[0]["timezones"]))
print(type(data[0]["flags"]))


<class 'dict'>
<class 'list'>
<class 'str'>
<class 'str'>
<class 'int'>
<class 'float'>
<class 'dict'>
<class 'dict'>
<class 'list'>
<class 'dict'>


SPRAWDZENIE WEWNATRZ WSZYSTKICH ZAGNIEZDZEN ILE ICH JEST I JAKIEGO SA TYPU ZA POMACA REKURENCYJNEJ FUNKCJI

In [5]:
def describe(val):
    # jeśli to słownik
    if isinstance(val, dict):
        return {k: describe(v) for k, v in val.items()}
    # jeśli to lista
    elif isinstance(val, list) and len(val) > 0:
        return {"list_of": describe(val[0])}
    # proste typy
    else:
        return type(val).__name__

# użycie:
sample = data[0]  # pierwszy rekord z JSON
field_types = {k: describe(v) for k, v in sample.items()}

print(field_types)

{'flags': {'png': 'str', 'svg': 'str', 'alt': 'str'}, 'name': {'common': 'str', 'official': 'str', 'nativeName': {'eng': {'official': 'str', 'common': 'str'}, 'jam': {'official': 'str', 'common': 'str'}}}, 'currencies': {'JMD': {'name': 'str', 'symbol': 'str'}}, 'capital': {'list_of': 'str'}, 'region': 'str', 'subregion': 'str', 'languages': {'eng': 'str', 'jam': 'str'}, 'area': 'float', 'population': 'int', 'timezones': {'list_of': 'str'}}


PRZEDSTAWIENIE STRURY I TYPY WEW. JSON PLIKU W SPOSOB UMOZLIWJACA NAOCZNA WERYFIKACJE

In [6]:


print(json.dumps(field_types, indent=2))


{
  "flags": {
    "png": "str",
    "svg": "str",
    "alt": "str"
  },
  "name": {
    "common": "str",
    "official": "str",
    "nativeName": {
      "eng": {
        "official": "str",
        "common": "str"
      },
      "jam": {
        "official": "str",
        "common": "str"
      }
    }
  },
  "currencies": {
    "JMD": {
      "name": "str",
      "symbol": "str"
    }
  },
  "capital": {
    "list_of": "str"
  },
  "region": "str",
  "subregion": "str",
  "languages": {
    "eng": "str",
    "jam": "str"
  },
  "area": "float",
  "population": "int",
  "timezones": {
    "list_of": "str"
  }
}


WALIDACJA Z PYDANTIC (weryfikacja czy typy i klasy sa prawidlowe)

In [8]:

from typing import List, Optional, Dict
from pydantic import BaseModel, ValidationError

class Flags(BaseModel):
    png: str
    svg: str
    alt: str

class NativeNameEntry(BaseModel):
    official: str
    common: str

class Name(BaseModel):
    common: str
    official: str
    nativeName: Optional[Dict[str, NativeNameEntry]]

class Currency(BaseModel):
    name: str
    symbol: str



class Country(BaseModel):
    name: Name
    flags: Flags
    currencies: Optional[Dict[str, Currency]]  # This line uses Currency
    capital: List[str]
    region: str
    subregion: Optional[str]
    languages: Dict[str, str]
    area: Optional[float]
    population: int
    timezones: List[str]


# Usage:
try:
    countries = [Country(**item) for item in data]
    print(f"Validated {len(countries)} countries successfully")
except ValidationError as e:
    print(f"Validation error: {e}")

Validated 250 countries successfully


SPRAWDZAM CZY W OPCJONALNYCH FIELDS SA JAKIEKOLWIEK BRAKI DANYCH

In [9]:
# Get all optional fields from your model
optional_fields = []
for field_name, field_info in Country.model_fields.items():
    if not field_info.is_required():
        optional_fields.append(field_name)

print(f"Optional fields: {optional_fields}")

# Check which optional fields are missing in each country
for i, item in enumerate(data):
    missing_optional = []
    for field in optional_fields:
        if field not in item or item[field] is None:
            missing_optional.append(field)

    if missing_optional:
        print(f"Country {i}: Missing optional data: {missing_optional}")


Optional fields: []


In [ ]:


    # TU WSTAWIAM PYDANTIC
    df = pd.json_normalize(data)
    # liczba krajów w danych
    print("Liczba krajów:", len(df))
    # podgląd pierwszych 5 wierszy
    print(df.head())


# df = pd.json_normalize(resp.json()) if resp.status_code == 200 else None
# print(df.head() if df is not None else resp.text)

Liczba krajów: 250
         capital    region           subregion      area  population  \
0     [Kingston]  Americas           Caribbean   10991.0     2961161   
1       [Moroni]    Africa      Eastern Africa    1862.0      869595   
2       [Harare]    Africa     Southern Africa  390757.0    14862927   
3  [Philipsburg]  Americas           Caribbean      34.0       40812   
4    [Vientiane]      Asia  South-Eastern Asia  236800.0     7275556   

     timezones                        flags.png                   flags.svg  \
0  [UTC-05:00]  https://flagcdn.com/w320/jm.png  https://flagcdn.com/jm.svg   
1  [UTC+03:00]  https://flagcdn.com/w320/km.png  https://flagcdn.com/km.svg   
2  [UTC+02:00]  https://flagcdn.com/w320/zw.png  https://flagcdn.com/zw.svg   
3  [UTC-04:00]  https://flagcdn.com/w320/sx.png  https://flagcdn.com/sx.svg   
4  [UTC+07:00]  https://flagcdn.com/w320/la.png  https://flagcdn.com/la.svg   

                                           flags.alt   name.common  ...  

In [ ]:
# sprawdzamy kilka pierwszych rekordów
for i, kraj in enumerate(data[:5]):
    try:
        # walidacja typów pól:
        assert isinstance(kraj.get("name"), dict)                  # nazwa kraju = słownik
        assert isinstance(kraj.get("capital"), list) or kraj.get("capital") is None  # stolica = lista lub None
        assert isinstance(kraj.get("population"), int)             # populacja = int
        assert isinstance(kraj.get("area"), (int, float))          # powierzchnia = int/float
        assert isinstance(kraj.get("currencies"), dict) or kraj.get("currencies") is None
        assert isinstance(kraj.get("languages"), dict) or kraj.get("languages") is None
        assert isinstance(kraj.get("region"), str) or kraj.get("region") is None      # region = string
        assert isinstance(kraj.get("subregion"), str) or kraj.get("subregion") is None # subregion = string
        assert isinstance(kraj.get("timezones"), list) or kraj.get("timezones") is None # strefy czasowe = lista
        assert isinstance(kraj.get("flags"), dict) or kraj.get("flags") is None        # flaga = dict

        print(f"✔ Rekord {i} poprawny")
    except AssertionError:
        print(f"❌ Błąd w rekordzie {i}: {kraj}")



✔ Rekord 0 poprawny
✔ Rekord 1 poprawny
✔ Rekord 2 poprawny
✔ Rekord 3 poprawny
✔ Rekord 4 poprawny
